 <div style='padding: 0; margin: 0; line-height: 1.2;'> <h1 style='margin: 0;'><b><font color='blue'>Performing ETL on Small Data</font></b></h1> <p style='margin: 0;'><b>1. Exploratory Analysis</b><br> <p style='margin: 0;'><b>2. Connecting to the Database</b><br>  - Creating a New Table.</p> <p style='margin: 0;'><b>3. Collecting Data from Different Sources</b><br>  - Uploading the Data from a CSV file<br><p style='margin: 0;'><b>4. Transforming the data</b><br><p style='margin: 0;'><b>5. Using API to Pull the Exchange Rate Data for Each Market from the Web</b><br><p style='margin: 0;'><b>6. Updating the Table in the Database</b><br></div>

In [11]:
#conda update jupyter
#%load_ext sql

In [12]:
#pip install psycopg2
#pip install sqlalchemy

In [13]:
import psycopg2
import pandas as pd
import sqlalchemy
from sqlalchemy import types
from sqlalchemy import create_engine

##<font color='blue'>Loading CSV in Dataframe</font>

In [15]:
df = pd.read_csv (r'C:\Users\Ashwini\Ably\transactions.csv')   


In [16]:
df.head()

,transaction_id,user,timestamp,tz,currency,value,cleared,country
0,4afd43a4-7833-4883-aa4f-1506cba532ce,47735,2/2/2022,Europe/Paris,USD,4999.8415,False,Canada
1,1777cfe1-0ab7-4c57-94f7-99c58d8a419c,76257,1/11/2022,Europe/Paris,GBP,3062.9059,False,Canada
2,f19e7c82-7b90-401d-a6af-fef1a7ecfbe7,30358,1/11/2022,US/Pacific,USD,3130.5393,False,France
3,d3e7b79b-fb99-434f-9a44-04df707f3741,24417,2/19/2022,US/Pacific,US Dollars,1091.9885,True,Canada
4,fe816fbb-9f09-4bd7-bdb4-46dfdbf4888a,56415,2/9/2022,US/Pacific,EUR,5427.2242,True,Portugal


In [17]:
df['timestamp'] = pd.to_datetime(df['timestamp'])

In [18]:
df['timestamp'] = df['timestamp'].dt.strftime('%d/%m/%Y')

In [19]:
print(df)

                           transaction_id   user   timestamp             tz  \
0    4afd43a4-7833-4883-aa4f-1506cba532ce  47735  02/02/2022   Europe/Paris   
1    1777cfe1-0ab7-4c57-94f7-99c58d8a419c  76257  11/01/2022   Europe/Paris   
2    f19e7c82-7b90-401d-a6af-fef1a7ecfbe7  30358  11/01/2022     US/Pacific   
3    d3e7b79b-fb99-434f-9a44-04df707f3741  24417  19/02/2022     US/Pacific   
4    fe816fbb-9f09-4bd7-bdb4-46dfdbf4888a  56415  09/02/2022     US/Pacific   
..                                    ...    ...         ...            ...   
865  69b5ff88-b2c7-4122-b560-74c968632e62  44868  11/01/2022     US/Eastern   
866  898881ce-d7fc-418b-9eb8-74d64d4a0a37  47819  20/01/2022     US/Eastern   
867  42aaae24-d50b-485b-a7ad-e58442aec87a  60535  20/01/2022  Europe/London   
868  539759c5-5303-4e48-b2a8-d6ba4797b15c  62664  13/01/2022     US/Eastern   
869  14235c77-8ae6-4b0c-8626-ca8f0b86f97a  94520  08/02/2022     US/Pacific   

       currency      value  cleared        country 

##<font color='blue'>Connecting to Postgresql Database</font>

In [14]:
try:
    conn = psycopg2.connect("dbname='mydb' user='postgres' host='localhost' password='Ashwini@12'")
except:
    print("Database Connection Error")
conn.autocommit = True
cursor = conn.cursor()

##<font color='blue'>Loading data into the Database</font>

In [20]:
cursor.execute("DROP TABLE IF EXISTS transaction_raw")

In [21]:
engine = create_engine('postgresql://postgres:Ashwini@12@localhost/mydb')
df.to_sql('transaction_raw',con= engine,if_exists='replace',index=False,
           dtype ={'transaction_id': sqlalchemy.types.VARCHAR(50),
                     'user' : sqlalchemy.types.INTEGER,
                     'timestamp' :sqlalchemy.types.Date,
                     'tz' :sqlalchemy.types.VARCHAR(length=20),
                     'currency':sqlalchemy.types.VARCHAR(length=10),
                     'value' :sqlalchemy.types.Numeric,
                     'cleared' :sqlalchemy.types.Boolean,
                     'country':sqlalchemy.types.VARCHAR(length=50)})

In [22]:
cursor.execute("DROP TABLE IF EXISTS transaction")
sql="""Create Table transaction as 
        Select * from transaction_raw
        where value is not null"""
cursor.execute(sql)

##<font color='blue'>Transforming the data</font>

In [23]:
sql = """SELECT table_name, column_name, data_type 
         FROM information_schema.columns
         WHERE table_name = 'transaction'"""
cursor.execute(sql)
for i in cursor.fetchall():
    print(i)

('transaction', 'transaction_id', 'character varying')
('transaction', 'user', 'integer')
('transaction', 'timestamp', 'date')
('transaction', 'tz', 'character varying')
('transaction', 'currency', 'character varying')
('transaction', 'value', 'numeric')
('transaction', 'cleared', 'boolean')
('transaction', 'country', 'character varying')


In [24]:
s = """ALTER TABLE transaction
       ALTER COLUMN value TYPE Numeric(10,2);
       """
cursor.execute(s)

In [25]:
up = """UPDATE transaction
        SET currency = 'EUR' 
        WHERE currency IN ('Euro')"""
cursor.execute(up)
upd = """UPDATE transaction
        SET currency = 'USD' 
        WHERE currency IN ('US Dollars')"""
cursor.execute(upd)

In [26]:
s0 = """UPDATE transaction
        SET currency = 'GBP' 
        WHERE currency isnull"""
cursor.execute(s0)

In [28]:
s2 = """UPDATE transaction
        SET country = 'United Kingdom'
        where country is null"""
cursor.execute(s2)

s3 = """select * from transaction where country is null
        """
cursor.execute(s3)
for i in cursor.fetchall():
    print(i)

##<font color='blue'>Extracting from API</font>

In [29]:
import json,requests

In [30]:
response = requests.get('https://v6.exchangerate-api.com/v6/435a57dc789ddcb46282a1b6/latest/GBP')

In [31]:
c_rates = response.text

In [32]:
rates_d = json.loads(c_rates)

In [33]:
rates_d

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1649289602,
 'time_last_update_utc': 'Thu, 07 Apr 2022 00:00:02 +0000',
 'time_next_update_unix': 1649376002,
 'time_next_update_utc': 'Fri, 08 Apr 2022 00:00:02 +0000',
 'base_code': 'GBP',
 'conversion_rates': {'GBP': 1,
  'AED': 4.802,
  'AFN': 115.9079,
  'ALL': 145.4647,
  'AMD': 625.6295,
  'ANG': 2.3405,
  'AOA': 587.6595,
  'ARS': 146.0767,
  'AUD': 1.7329,
  'AWG': 2.3405,
  'AZN': 2.219,
  'BAM': 2.3451,
  'BBD': 2.6151,
  'BDT': 111.6159,
  'BGN': 2.3448,
  'BHD': 0.4916,
  'BIF': 2658.8169,
  'BMD': 1.3076,
  'BND': 1.7785,
  'BOB': 8.9907,
  'BRL': 6.1158,
  'BSD': 1.3076,
  'BTN': 98.9244,
  'BWP': 15.132,
  'BYN': 3.8194,
  'BZD': 2.6151,
  'CAD': 1.6366,
  'CDF': 2604.6342,
  'CHF': 1.22,
  'CLP': 1029.0028,
  'CNY': 8.3177,
  'COP': 4882.2612,
  'CRC': 862.9364,
  'CUP': 31.3816,
  'CVE': 132.21,
  'CZK': 

##<font color='blue'>Updating the Table</font>

In [34]:
cursor.execute("DROP TABLE IF EXISTS exchange_rate")

In [35]:
sql4 = """create table exchange_rate(currency varchar, rate Numeric(10,2))"""
cursor.execute(sql4)

In [36]:
sql6 = """INSERT INTO exchange_rate VALUES(%s, %s)"""

for record in rates_d["conversion_rates"].items():
    cursor.execute(sql6, [record[0], record[1]])


In [37]:
sql9 = """select * from exchange_rate """
cursor.execute(sql9)
for i in cursor.fetchall():
    print(i)

('GBP', Decimal('1.00'))
('AED', Decimal('4.80'))
('AFN', Decimal('115.91'))
('ALL', Decimal('145.46'))
('AMD', Decimal('625.63'))
('ANG', Decimal('2.34'))
('AOA', Decimal('587.66'))
('ARS', Decimal('146.08'))
('AUD', Decimal('1.73'))
('AWG', Decimal('2.34'))
('AZN', Decimal('2.22'))
('BAM', Decimal('2.35'))
('BBD', Decimal('2.62'))
('BDT', Decimal('111.62'))
('BGN', Decimal('2.34'))
('BHD', Decimal('0.49'))
('BIF', Decimal('2658.82'))
('BMD', Decimal('1.31'))
('BND', Decimal('1.78'))
('BOB', Decimal('8.99'))
('BRL', Decimal('6.12'))
('BSD', Decimal('1.31'))
('BTN', Decimal('98.92'))
('BWP', Decimal('15.13'))
('BYN', Decimal('3.82'))
('BZD', Decimal('2.62'))
('CAD', Decimal('1.64'))
('CDF', Decimal('2604.63'))
('CHF', Decimal('1.22'))
('CLP', Decimal('1029.00'))
('CNY', Decimal('8.32'))
('COP', Decimal('4882.26'))
('CRC', Decimal('862.94'))
('CUP', Decimal('31.38'))
('CVE', Decimal('132.21'))
('CZK', Decimal('29.35'))
('DJF', Decimal('232.38'))
('DKK', Decimal('8.95'))
('DOP', Decimal(

In [38]:
sql10 = """SELECT table_name, column_name, data_type 
           FROM information_schema.columns
           WHERE table_name = 'exchange_rate'"""
cursor.execute(sql10)
for i in cursor.fetchall():
    print(i)

('exchange_rate', 'currency', 'character varying')
('exchange_rate', 'rate', 'numeric')
